### Import Libraries

In [1]:
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn
from sklearn.metrics import accuracy_score

### Import dataset

In [3]:
train_dataset = pd.read_csv('./ILDC/ILDC_single/train_dataset.csv')
dev_dataset = pd.read_csv('./ILDC/ILDC_single/dev_dataset.csv')
test_dataset = pd.read_csv('./ILDC/ILDC_single/test_dataset.csv')
print(f'Train Dataset: {train_dataset.shape}')
print(f'Dev Dataset: {dev_dataset.shape}')
print(f'Test Dataset: {test_dataset.shape}')

Train Dataset: (5082, 2)
Dev Dataset: (994, 2)
Test Dataset: (1517, 2)


In [4]:
texts = []
for text in train_dataset['text']:
    texts.append(' '.join(text.split(' ')[-500:]))
train_dataset['text'] = texts
texts = []
for text in test_dataset['text']:
    texts.append(' '.join(text.split(' ')[-500:]))
test_dataset['text'] = texts

### Train Model (RoBERTa)




In [5]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 5
model_args.learning_rate = 1e-5
model_args.overwrite_output_dir = True

In [6]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args = model_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [7]:
model.train_model(train_dataset)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5082 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

(3180, 0.6097961275832459)

In [8]:
result, model_outputs, wrong_predictions = model.eval_model(test_dataset, acc = accuracy_score)
result

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1517 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/190 [00:00<?, ?it/s]

{'mcc': 0.086109033435398,
 'tp': 320,
 'tn': 501,
 'fp': 254,
 'fn': 442,
 'auroc': 0.5736802767203769,
 'auprc': 0.5831571823318277,
 'acc': 0.5411997363216875,
 'eval_loss': 0.8624204074081622}

### Train Model (bert-base-uncased)

In [9]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 5
model_args.learning_rate = 1e-5
model_args.overwrite_output_dir = True

In [10]:
model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, args = model_args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
model.train_model(train_dataset)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5082 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

(3180, 0.5777484593901244)

In [12]:
result, model_outputs, wrong_predictions = model.eval_model(test_dataset, acc = accuracy_score)
result

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1517 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/190 [00:00<?, ?it/s]

{'mcc': 0.06349393237272088,
 'tp': 128,
 'tn': 662,
 'fp': 93,
 'fn': 634,
 'auroc': 0.5637491091759226,
 'auprc': 0.5500329868788982,
 'acc': 0.5207646671061306,
 'eval_loss': 0.9968703702876442}

### Train Model (legal-bert-base-uncased)




In [13]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 5
model_args.learning_rate = 1e-5
model_args.overwrite_output_dir = True

In [14]:
model = ClassificationModel('bert', 'nlpaueb/legal-bert-base-uncased', num_labels=2, args = model_args)

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [15]:
model.train_model(train_dataset)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5082 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

(3180, 0.5682440187946056)

In [16]:
result, model_outputs, wrong_predictions = model.eval_model(test_dataset, acc = accuracy_score)
result

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1517 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/190 [00:00<?, ?it/s]

{'mcc': 0.13426757564125727,
 'tp': 289,
 'tn': 563,
 'fp': 192,
 'fn': 473,
 'auroc': 0.6008134744746311,
 'auprc': 0.5938960727671758,
 'acc': 0.5616348055372445,
 'eval_loss': 0.876118655894932}

### Train Model (saibo/legal-roberta-base)

In [17]:
model_args = ClassificationArgs()

model_args.num_train_epochs = 5
model_args.learning_rate = 1e-5
model_args.overwrite_output_dir = True

In [18]:
model = ClassificationModel('roberta', 'saibo/legal-roberta-base', num_labels=2, args = model_args)

Some weights of the model checkpoint at saibo/legal-roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at saibo/legal-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.wei

In [19]:
model.train_model(train_dataset)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5082 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/636 [00:00<?, ?it/s]

(3180, 0.5994749567043857)

In [20]:
result, model_outputs, wrong_predictions = model.eval_model(test_dataset, acc = accuracy_score)
result

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1517 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/190 [00:00<?, ?it/s]

{'mcc': 0.12517373975758742,
 'tp': 136,
 'tn': 685,
 'fp': 70,
 'fn': 626,
 'auroc': 0.5849324711894457,
 'auprc': 0.5939263006390844,
 'acc': 0.5411997363216875,
 'eval_loss': 0.9848184685958059}